## Gaussian Mixture ##

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import multivariate_normal as Nd

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

A Gaussian mixture is a [mixture distribution](https://en.wikipedia.org/wiki/Mixture_distribution) of $d$ normal  distributions. 

A multivariate normal distribution is defined in  `scipy.stats`  as : V=Nd(mean=m, cov=Sigma), where $m=(m[0,], m[1], \ldots m[d-1])^T$ is the vector of means, and  $\Sigma\in\mathbb{R}^{d\times d}$, the covariance matrix.

We define a mixture of 4 bivariate normal distributions. For each bivariate distribution we set the mean vector, 
the standard deviation vector and the correlation coefficient of the corresponding random variables associated to that bivariate distribution:

In [ ]:
pr=[0.2, 0.4, 0.15, 0.25]# the list of probabilities defining the Gaussian mixture
means=np.array([[0.5, 1.], [0.9, 2.7], [2., 3.],[2.5, 2]])
sigmas=np.array([[0.7, 0.6],   [0.7, 0.9],  [0.41, 0.23 ],[0.53, 0.35]]) # pairs of standard deviations
rho=np.array([0.0, -0.67,-0.76,0.5])# coefficients of correlation

In [ ]:
def Covariance(m, sigma, r):# defines the covariance matrix matrix for a a normal bivariate  (X,Y)~N(m,Sigma)
    covar= r*sigma[0]*sigma[1]
    return np.array([[sigma[0]**2, covar], [covar, sigma[1]**2]])  
    

The four covariance matrices are stored in a 3D `numpy.array`, `Sigma`:

In [ ]:
Sigma=np.zeros((2,2,4), float)
#Sigma[:,:, k] is the covariance matrix of the (k+1)^{th} bivariate normal distribution of  the mixture

for k in range(4):
    Sigma[:,:,k]=Covariance(means[k, :], sigmas[k,:], rho[k])
    

`mixtureDensity` defines the probability density function of the Gaussian mixture:

In [ ]:
def mixtureDensity(x,y, pr, means, Sigma):
    pos=np.empty(x.shape + (2,))# if  x.shape is (m,n) then  pos.shape is (m,n,2)
    pos[:, :, 0] = x; pos[:, :, 1] = y 
    z=np.zeros(x.shape)
    for k in range(4) :
        z=z+pr[k]*Nd.pdf(pos, mean=means[k,:], cov=Sigma[:,:, k])
    return z     

The 3D `numpy.array`, pos, is defined in concordance to the  definition in `scipy.stats` of a 
[multivariate normal distribution](http://docs.scipy.org/doc/scipy-dev/reference/generated/scipy.stats.multivariate_normal.html). 

### Matplotlib plot of the Gaussian mixture probability density function ###

In [ ]:
plt.rcParams['figure.figsize'] = (10.0, 6.0)

We define a custom colormap to plot the mixture's pdf:

In [ ]:
xx=np.linspace(-1, 4, 100)
yy=np.linspace(0, 4.5, 100)
x,y=np.meshgrid(xx,yy)
z=mixtureDensity(x, y, pr,  means, Sigma)

fig=plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Probability density function of the Gaussian Mixture')
ax.view_init(elev=35, azim=-90)
ax.plot_surface(x, y, z, rstride=1, cstride=1, cmap=cm.RdBu,
        linewidth=0, antialiased=False)

### Plotly plot of the probability density function of the Gaussian mixture ###

To see this surface from different points of view we generate an interactive 
   Plotly plot:

In [ ]:
mountain_cs=[[0.0, '#32924c'], # a custom colorscale 
 [0.1, '#52a157'],
 [0.2, '#74b162'],
 [0.3, '#94c06d'],
 [0.4, '#b6d079'],
 [0.5, '#d7de84'],
 [0.6, '#c9c370'],
 [0.7, '#bba65b'],
 [0.8, '#ad8a47'],
 [0.9, '#9f6d32'],
 [1.0, '#91511e']]

In [ ]:
import plotly.plotly as py
from plotly.graph_objs import *

In [ ]:
trace1= Surface(
       z=z,
       x=x,
       y=y, 
       colorscale=mountain_cs
    )

In [ ]:
axis = dict(
showbackground=True, 
backgroundcolor="rgb(230, 230,230)",
gridcolor="rgb(255, 255, 255)",      
zerolinecolor="rgb(255, 255, 255)",  
    )

layout = Layout(
         title="" , 
         autosize=False,
         width=600,
         height=500,
         scene=Scene(  
             xaxis=XAxis(axis),
             yaxis=YAxis(axis),
             zaxis=ZAxis(axis), 
            )
        )

fig = Figure(data=[trace1], layout=layout)
py.sign_in('empet', 'my_api_key')
py.plot(fig, filename='Gaussian-Mixture-P-Distrib', world_readable=True)

### Contour plot and a cloud of points as a sample from the Gaussian mixture ###